In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import astropy.constants as cte
import astropy.units as un
from astropy.cosmology import Planck15 as cosmo
import emcee
import scipy
from scipy import optimize as op
import corner
import os
from datetime import datetime

In [2]:
os.environ["OMP_NUM_THREADS"]="4"

In [3]:
c=cte.c
Msun=cte.M_sun

In [4]:
file = open("/Users/mariajesusfloresmoraga/Desktop/DSFG/SPT.csv")
csvreader = csv.reader(file)
header = []
header = next(csvreader)

rows = []
for row in csvreader:
        rows.append(row)

ID= []

z=[]

S3=[]
errS3=[]

S2=[]
errS2=[]

S1=[]
errS1=[]

S870=[]
errS870=[]

S500=[]
errS500=[]

S350=[]
errS350=[]

S250=[]
errS250=[]

S160=[]
errS160=[]

S100=[]
errS100=[]

for j in rows:   
    
    ID.append(j[0])
    z.append(float(j[1]))
    S3.append(float(j[2]))
    errS3.append(float(j[3]))
    S2.append(float(j[4]))
    errS2.append(float(j[5]))
    S1.append(float(j[6]))
    errS1.append(float(j[7]))
    S870.append(float(j[8]))
    errS870.append(float(j[9]))
    S500.append(float(j[10]))
    errS500.append(float(j[11]))
    S350.append(float(j[12]))
    errS350.append(float(j[13]))
    S250.append(float(j[14]))
    errS250.append(float(j[15]))
    S160.append(float(j[16]))
    errS160.append(float(j[17]))
    S100.append(float(j[18]))
    errS100.append(float(j[19]))
   

In [5]:
v__=np.array([870,500,350])*un.um
v_=((c.to("um/s"))/v__).to("Hz")
v=v_.value
l__=np.array([3.2,2,1.4])*un.mm
l_=((c.to("mm/s"))/l__).to("Hz")
l=l_.value
vf=np.concatenate((l,v))
print((vf*un.Hz).to("GHz"))

[ 93.68514313 149.896229   214.13747    344.58903218 599.584916
 856.54988   ] GHz


In [21]:
def clean(j):
    
    nlista=[]

    for h in j:
        
        if h==0 or np.isnan(h):
                
            nlista.append(np.delete(vf,j.index(h)))
        
        else:
            
            nlista.append(vf)
                
    return np.array(nlista)

In [25]:
def tau(z,M_,d_,vf): 
    print(vf)
    vf=vf*un.Hz
    d_=d_*un.kpc
    v0=353*un.GHz
    b=2
    M=Msun * 10** M_
    k0=0.15*(un.m**2/un.kg) 
    tau=k0*(vf/v0)**b *(z+1)**b * M/(np.pi*(d_/2)**2)
    return tau.to("")

In [8]:
def P(vf,T,z):
    vf=vf*un.Hz
    T=T*un.K
    h=cte.h
    K=cte.k_B
    A=2*h/(c**2)
    a=(h*vf*(1+z)/(K*T))
    B=vf**3/(np.e**(a.value) -1)
    P=(A*B).to("mJy")
    return P

In [9]:
def modelSv(vf,z,T,M_,d_):
    
    b=2
    v0=353*un.GHz
    D=cosmo.angular_diameter_distance(z)
    M=Msun*(10**M_)
    
    TAU=tau(z,M_,d_,vf)
    PL=P(vf,T,z)
    
    vf=vf*un.Hz
    T=T*un.K
    d_=d_*un.kpc
    
    O=np.pi*(d_/2)**2 * (D**-2)
    
    S_=O*(1-np.e**-TAU)*PL
    S=S_.to("mJy")
    return S.value

In [10]:
def A(z,Tc,M_,d_,vf): 


    v0=353*un.GHz
    b=2
    
    Tc=Tc*un.K
    Tvariable=(np.linspace(Tc.value,1000,10000)) *un.K
    
    integrales=[]
    
    for i in vf:
        ARG = modelSv(i,z,Tvariable,M_,d_)*un.mJy *(Tvariable)**-7
        integrales.append(scipy.integrate.trapz(ARG,x=Tvariable))
    
    return integrales

In [11]:
def modelSobs(vf,z,Tc,M_,d_): #Como SPT no entrega Tc tomamos Tc= T-10]
    
    b=2
    v0=353*un.GHz
    D=cosmo.angular_diameter_distance(z)
    TCMB=2.725*un.K
    
    INTEGRAL=A(z,Tc,M_,d_,vf)
    TAU=tau(z,M_,d_,vf)
    
    h=cte.h
    K=cte.k_B
    vf=vf*un.Hz
    d_=d_*un.kpc
    Tc=Tc*un.K
    O=np.pi*(d_/2)**2 * (D**-2)
    model_=[]
    for j in range(len(vf)):
        M1=6 * Tc**6 * INTEGRAL[j]
    
        M2=O * np.e**-TAU * (2*h)/(c**2) * (vf[j]**3)/(np.e**(h*vf[j]/(K*TCMB))-1)
        model_.append(((M1+M2)[0]).value)
        
    model=model_*un.mJy

    return model.value

In [12]:
frecsT_l=[]
errfT_l=[]
for m in range (len(rows)-1):
    frec=[S3[m],S2[m],S1[m],S870[m],S500[m],S350[m]]
    errf=[errS3[m],errS2[m],errS1[m],errS870[m],errS500[m],errS350[m]]
    frecsT_l.append(frec)
    errfT_l.append(errf)

In [27]:
def ml():
    
    x = np.linspace(90,2000,100)*un.GHz
    x = x.to("Hz")
    x = x.value
    
    LT_S=[]
    
    LT_PLnolim=[]
    
    LT_PLlim=[]
    
    
    for j in frecsT_l:
        print(j)
       
        popt,pconv = scipy.optimize.curve_fit(modelSv,clean,np.array(j),[3.,50.,10.,4.],bounds=([1,10,8,2],[10,100,11,4])) 
    
        z_fit,T_fit,M_fit,d_fit = popt
    

        pm = [z_fit,T_fit,M_fit,d_fit]

        y = modelSv(x,pm[0],pm[1],pm[2],pm[3])
                 
        L= [ID[frecsT_l.index(j)],z_fit,T_fit,M_fit,d_fit]
        
        LT_S.append(L)
            

        
    for j in frecsT_l:
       
        popt,pconv = scipy.optimize.curve_fit(modelSobs,clean,np.array(j),[3.,50.,10.,4.],bounds=([1,10,8,2],[10,100,11,4])) 
    
        z_fit,T_fit,M_fit,d_fit = popt
    

        pm = [z_fit,T_fit,M_fit,d_fit]

        y = modelSv(x,pm[0],pm[1],pm[2],pm[3])
                 
        L= [ID[frecsT_l.index(j)],z_fit,T_fit,M_fit,d_fit]
        
        LT_PLnolim.append(L)
        
        
    for j in frecsT_l:
       
        popt,pconv = scipy.optimize.curve_fit(modelSobs,clean(j),np.array(j),[3.,35.,10.,4.],bounds=([1,20,8,2],[10,50,11,4])) 
    
        z_fit,T_fit,M_fit,d_fit = popt
    

        pm = [z_fit,T_fit,M_fit,d_fit]

        y = modelSv(x,pm[0],pm[1],pm[2],pm[3])
                 
        L= [ID[frecsT_l.index(j)],z_fit,T_fit,M_fit,d_fit]
        
        LT_PLlim.append(L)
    
    z=[]
    T=[]
    M=[]
    d=[]
        
    for a,b,c in zip(LT_S,LT_PLnolim,LT_PLlim):
        z_=[]
        z_.append(a[0]) #para el nombre
        z_.append(a[1])
        z_.append(b[1])
        z_.append(c[1])
        z.append(z_)
        
        T_=[]
        T_.append(a[0]) #para el nombre
        T_.append(a[2])
        T_.append(b[2])
        T_.append(c[2])
        T.append(T_) 
        
        M_=[]
        M_.append(a[0]) #para el nombre
        M_.append(a[3])
        M_.append(b[3])
        M_.append(c[3])
        M.append(M_) 
        
        d_=[]     
        d_.append(a[0]) #para el nombre
        d_.append(a[3])
        d_.append(b[3])
        d_.append(c[3])
        d.append(d_) 
        
                
    return  (LT_S,LT_PLlim,LT_PLnolim) 

In [26]:
ml()

[0.47, 3.1, 10.7, 50.3, 202.0, 283.5]
<function clean at 0x104aa0dd0>


TypeError: unsupported operand type(s) for *: 'function' and 'Unit'